In [234]:
import numpy as np
import pandas as pd
import os

import tensorflow as tf
import numpy as np
from IPython.display import YouTubeVideo

import requests
import json

import re

import torch
import torch.nn as nn
import torch.nn.functional as F
import math

from scipy.signal import peak_prominences
from scipy.signal import find_peaks

import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
import itertools
import random


In [2]:
path = r"/Users/scottmerrill/Documents/UNC/MultiModal/VMR/Youtube8m"
filenames = os.listdir(path + '/video')
file_name = filenames[0]

### 1.  DataLoader

In [3]:
class VideoAudioDataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.filenames = os.listdir(os.path.join(path, 'video'))
    
    def __len__(self):
        return len(self.filenames)
    
    def __getitem__(self, idx):
        filename = self.filenames[idx]
        video_data = np.load(os.path.join(self.path, 'video', filename))
        audio_data = np.load(os.path.join(self.path, 'audio', filename))
        video_data = video_data[:, :1024]
        return video_data, audio_data

### 2. Transformer Class

In [4]:
class Transformer(nn.Module):
    def __init__(self, input_dim=1024, embed_dim=512, num_heads=8, num_layers=2, max_seq_len=50):
        super().__init__()
        self.input_proj = nn.Linear(input_dim, embed_dim)  # Project input to embedding dim
        self.pos_encoder = self._generate_sinusoidal_positional_encoding(max_seq_len, embed_dim)
        self.transformer = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads), num_layers=num_layers)
        self.output_proj = nn.Linear(embed_dim, embed_dim)  # Project to final embedding

    def _generate_sinusoidal_positional_encoding(self, max_len, embed_dim):
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))
        pe = torch.zeros(max_len, embed_dim)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)  # Shape: (1, max_len, embed_dim)

    def forward(self, x, mask):
        x = self.input_proj(x)  # Shape: (seq_len, embed_dim)
        seq_len = x.size(0)
        x = x + self.pos_encoder[:, :seq_len, :].squeeze(0).to(x.device)
        x = self.transformer(x.unsqueeze(1), src_key_padding_mask=mask).squeeze(1)
        x = x.mean(dim=0)  # Aggregate sequence to fixed-size embedding
        return self.output_proj(x)

### Optical Flow Class

In [195]:
class OpticalFlowProcessor:
    def __init__(self, method='video', window_size=20, max_segments=10, min_frames=10):
        self.method = method
        self.window_size = window_size
        self.max_segments = max_segments
        self.min_frames = min_frames

    def get_of_ranks(self, rgb, audio):
        flow = self._compute_flow(rgb, audio)
        segments = self._optical_flow_segments(flow)
        ranks = self._rank_averages(self._compute_segment_means(segments, flow))
        return ranks

    def get_best_worst_flow(self, rgb, audio):
        flow = self._compute_flow(rgb, audio)
        segments = self._optical_flow_segments(flow)
        ranks = self._rank_averages(self._compute_segment_means(segments, flow))
        return self._extract_best_worst_segments(segments, ranks)

    def _compute_flow(self, rgb, audio):
        if self.method == 'video':
            return self._moving_average(self._calculate_optical_flow_euclidean(rgb))
        elif self.method == 'audio':
            return self._moving_average(self._calculate_optical_flow_euclidean(audio))
        else:
            raise ValueError("Method must be 'video' or 'audio'")

    @staticmethod
    def _calculate_optical_flow_euclidean(embedding_seq):
        return np.linalg.norm(embedding_seq[1:] - embedding_seq[:-1], axis=1)

    @staticmethod
    def _moving_average(arr, window_size=5):
        return np.convolve(arr, np.ones(window_size) / window_size, mode='valid')

    def _optical_flow_segments(self, optical_flow):
        peaks, _ = find_peaks(optical_flow)
        prominences = peak_prominences(optical_flow, peaks)[0]
        peak_index = peaks[np.argsort(prominences)[-self.max_segments:]]
        peak_index = self._merge_intervals(np.sort(peak_index))
        return np.insert(np.append(peak_index, len(optical_flow)), 0, 0)

    def _merge_intervals(self, arr):
        merged = [arr[0]]
        for i in range(1, len(arr)):
            if arr[i] - merged[-1] >= self.min_frames:
                merged.append(arr[i])
        return np.array(merged)

    @staticmethod
    def _compute_segment_means(segments, values):
        return [values[start:end].mean() if start < end else 0 for start, end in zip(segments[:-1], segments[1:])]

    @staticmethod
    def _rank_averages(averages):
        sorted_indices = np.argsort(averages)[::-1]
        ranks = np.zeros_like(sorted_indices) + 1
        for rank, idx in enumerate(sorted_indices):
            ranks[idx] = rank + 1
        return ranks

    def _extract_best_worst_segments(self, segments, ranks):
        top_start, top_end = segments[np.where(ranks == 1)[0][0]], segments[np.where(ranks == 1)[0][0] + 1]
        bottom_start, bottom_end = segments[np.where(ranks == max(ranks))[0][0]], segments[np.where(ranks == max(ranks))[0][0] + 1]
        return (top_start, top_end), (bottom_start, bottom_end)

### Script Functions

In [196]:
def collate_fn(batch, processor):
    video_batch, audio_batch = zip(*batch)
    video_batch = [torch.tensor(v, dtype=torch.float32) for v in video_batch]
    audio_batch = [torch.tensor(a, dtype=torch.float32) for a in audio_batch]
    flow_ranks = [processor.get_of_ranks(video_batch[i], audio_batch[i]) for i in range(len(video_batch))]
    return video_batch, audio_batch, flow_ranks

def get_dataloader(path, batch_size=32, shuffle=True, method='video', window_size=20):
    dataset = VideoAudioDataset(path)
    processor = OpticalFlowProcessor(method=method, window_size=window_size)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, collate_fn=lambda batch: collate_fn(batch, processor))

def perform_feature_padding(video_features, audio_features, start_segment, end_segment, max_seq_len):
    vf = torch.tensor(video_features[start_segment:end_segment,:])
    af = torch.tensor(audio_features[start_segment:end_segment,:])

    pvf = torch.zeros(max_seq_len, 1024)
    pvf[:vf.shape[0], :] = vf

    paf = torch.zeros(max_seq_len, 128)
    paf[:af.shape[0], :] = af

    # Create mask (True for padding positions)
    mask = torch.arange(max_seq_len) >= vf.shape[0]
    mask = mask.unsqueeze(0)  # Convert to 2D (batch_size=1, seq_len)
    return pvf, paf, mask

# Function to find pairs with approximately equal differences
def find_matching_index_pairs(array1, array2, tolerance=5):
    # Calculate differences in array1 and array2
    array1_diffs = np.diff(array1)
    array2_diffs = np.diff(array2)

    matching_pairs = []

    # Loop through differences in array1
    for i, diff1 in enumerate(array1_diffs):
        # Find pairs of consecutive indices in array2 with similar differences
        for j, diff2 in enumerate(array2_diffs):
            if abs(diff1 - diff2) <= tolerance:  # If the difference is within the tolerance
                matching_pairs.append(((i, i + 1), (j, j + 1), diff1, diff2))

    return matching_pairs


def get_similar_length_segments(positive_segments, negative_segments, tolerance = 5):
    
    while True:
        matching_indexes = find_matching_index_pairs(positive_segments, negative_segments, tolerance=tolerance)
        tolerance += 5
        if len(matching_indexes) > 0:
            break
            
    # sample randomly for all segments within the tolerance band
    pos_segment, negative_segment, pos_time, neg_time = matching_indexes[np.random.randint(0, len(matching_indexes))]
    
    return pos_segment, negative_segment, pos_time, neg_time

def get_positive_negative_embeddings(filenames, tolerance=5):

    positive_record_file = np.random.choice(filenames)
    negative_record_file = np.random.choice(filenames)

    positive_rgb, positive_audio = sample_dataset(positive_record_file)
    negative_rgb, negative_audio = sample_dataset(negative_record_file)

    optical_flow_pos = calculate_optical_flow_euclidean(positive_rgb)
    optical_flow_pos = moving_average(optical_flow_pos, window_size=20)

    optical_flow_neg = calculate_optical_flow_euclidean(negative_rgb)
    optical_flow_neg = moving_average(optical_flow_neg, window_size=20)

    positive_segments = optical_flow_segments(optical_flow_pos)
    negative_segments = optical_flow_segments(optical_flow_neg)

    pos_segment, negative_segment, pos_time, neg_time = get_similar_length_segments(positive_segments, negative_segments, tolerance = 5)

    # converting segment index to time in seconds
    pos_start, pos_end = pos_segment
    pos_start = positive_segments[pos_start]
    pos_end = positive_segments[pos_end]
    print(pos_start, pos_end)

    # retrieving segment and padding it appropriately
    pos_video, pos_audio, pos_mask = perform_feature_padding(positive_rgb, positive_audio, pos_start, pos_end, max_seq_len)

    # computing embedding
    positive_video_embedding = video_model(pos_video, pos_mask)
    positive_audio_embedding = audio_model(pos_audio, pos_mask)

    # converting segment index to time in seconds
    neg_start, neg_end = negative_segment
    neg_start = negative_segments[neg_start]
    neg_end = negative_segments[neg_end]

    # retrieving segment and padding it appropriately
    neg_video, neg_audio, neg_mask = perform_feature_padding(negative_rgb, negative_audio, neg_start, neg_end, max_seq_len)

    # computing embedding
    negative_video_embedding = video_model(neg_video, neg_mask)
    negative_audio_embedding = audio_model(neg_audio, neg_mask)
    return positive_video_embedding, positive_audio_embedding, negative_video_embedding, negative_audio_embedding

def save_checkpoint(model, optimizer, epoch, filename):
    """Saves model and optimizer state dict."""
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)
    print(f"Checkpoint saved at epoch {epoch} to {filename}")


In [197]:
def get_positive_negative_embeddings(video_batch, tolerance=5):

    positive_record_file = np.random.choice(filenames)
    negative_record_file = np.random.choice(filenames)

    positive_rgb, positive_audio = sample_dataset(positive_record_file)
    negative_rgb, negative_audio = sample_dataset(negative_record_file)

    optical_flow_pos = calculate_optical_flow_euclidean(positive_rgb)
    optical_flow_pos = moving_average(optical_flow_pos, window_size=20)

    optical_flow_neg = calculate_optical_flow_euclidean(negative_rgb)
    optical_flow_neg = moving_average(optical_flow_neg, window_size=20)

    positive_segments = optical_flow_segments(optical_flow_pos)
    negative_segments = optical_flow_segments(optical_flow_neg)

    pos_segment, negative_segment, pos_time, neg_time = get_similar_length_segments(positive_segments, negative_segments, tolerance = 5)

    # converting segment index to time in seconds
    pos_start, pos_end = pos_segment
    pos_start = positive_segments[pos_start]
    pos_end = positive_segments[pos_end]
    print(pos_start, pos_end)

    # retrieving segment and padding it appropriately
    pos_video, pos_audio, pos_mask = perform_feature_padding(positive_rgb, positive_audio, pos_start, pos_end, max_seq_len)

    # computing embedding
    positive_video_embedding = video_model(pos_video, pos_mask)
    positive_audio_embedding = audio_model(pos_audio, pos_mask)

    # converting segment index to time in seconds
    neg_start, neg_end = negative_segment
    neg_start = negative_segments[neg_start]
    neg_end = negative_segments[neg_end]

    # retrieving segment and padding it appropriately
    neg_video, neg_audio, neg_mask = perform_feature_padding(negative_rgb, negative_audio, neg_start, neg_end, max_seq_len)

    # computing embedding
    negative_video_embedding = video_model(neg_video, neg_mask)
    negative_audio_embedding = audio_model(neg_audio, neg_mask)
    return positive_video_embedding, positive_audio_embedding, negative_video_embedding, negative_audio_embedding

In [198]:
max_seq_len = 100


def get_segmentd_embeddings(video_model, audio_model, vid, aud):
    vid_segment_embeddings = []
    

    of = OpticalFlowProcessor()
    flow = of._compute_flow(vid, aud)
    segments = of._optical_flow_segments(flow)

    vid_segment_embeddings = []
    aud_segment_embeddings = []
    for i in range(1, len(segments)):
        start = segments[i-1]
        end = segments[i]

        vid_emb, aud_emb, mask = perform_feature_padding(vid, aud, start, end, max_seq_len)
        
        vid_segment_embeddings.append(video_model(vid_emb, mask))
        aud_segment_embeddings.append(audio_model(aud_emb, mask))
    return vid_segment_embeddings, aud_segment_embeddings

In [199]:
num_heads =1
num_layers=1
audio_model = Transformer(input_dim=128, embed_dim=256, num_heads=num_heads, num_layers=num_layers, max_seq_len=max_seq_len)
video_model = Transformer(input_dim=1024, embed_dim=256, num_heads=num_heads, num_layers=num_layers, max_seq_len=max_seq_len)

def get_batch_embeddings(video_model, audio_model, video_batch, audio_batch):
    # We precompute the segment embeddings in each batch.  We do this once and then proceed to processing batch
    batch_vid_embeddings = []
    batch_aud_embeddings = []
    for i in range(len(video_batch)):
        vid = video_batch[i]
        aud = audio_batch[i]
        vid_sgmt_emb, aud_sgmt_emb = get_segmentd_embeddings(video_model, audio_model, vid, aud)
        batch_vid_embeddings.extend(vid_sgmt_emb)
        batch_aud_embeddings.extend(aud_sgmt_emb)
        
    # Shape will by (total segments X embedding dim)
    # total segments is clip dependent
    batch_aud_embeddings = torch.stack(batch_aud_embeddings)
    batch_vid_embeddings = torch.stack(batch_vid_embeddings)
    
    # MAKE SURE VECTORS ARE NORMALIZED FIRST idk if I want to do here or later..
    batch_aud_embeddings = torch.nn.functional.normalize(batch_aud_embeddings, p=2, dim=1)
    batch_vid_embeddings = torch.nn.functional.normalize(batch_vid_embeddings, p=2, dim=1)

    return batch_aud_embeddings, batch_vid_embeddings

In [200]:
def get_intermodal_loss(batch_vid_embeddings, batch_aud_embeddings, k=5, min_val=0):
    # batch_vid_embeddings and  batch_aud_embeddings should already be normalized so 
    # multiplying them is a similarity metric
    
    # convert simliarity to distance by (-1) >> high value indicates the distance between the samples is long
    dist_xy = (-1) *torch.matmul(batch_vid_embeddings, batch_aud_embeddings.T)
    
    positive_pairs = torch.diag(dist_xy)

    # Get non-diagonal elements (negative examples)
    # First, create a mask for non-diagonal elements
    mask = ~torch.eye(dist_xy.size(0), dtype=torch.bool)

    # Apply the mask to extract non-diagonal elements
    negative_pairs = dist_xy[mask]
    
    topk_pos_values, _ = torch.topk(positive_pairs.flatten(), k)
    topk_neg_values, _ = torch.topk(negative_pairs.flatten(), k)
    
    # max accross each pos/neg pair
    loss = torch.max(topk_pos_values - topk_neg_values, torch.tensor(min_val))
    return torch.mean(loss)

## Training Loop

In [189]:
### Parameters

In [248]:
lambda1 = 0.33
lambda2 = 0.33
lambda3 = 0.33

lr = 1e-4

# Define the Adam optimizer for the audio model
audio_optimizer = optim.Adam(audio_model.parameters(), lr=lr)

# Define the Adam optimizer for the video model
video_optimizer = optim.Adam(video_model.parameters(), lr=lr)

batch_size = 2
window_size = 20 # for optical flow smoothing (ie 20 frame mavg flow)

dataloader = get_dataloader(path, batch_size=batch_size, shuffle=True, method='video', window_size=window_size)
num_flow_matching = 10
k = 20
margin = 0.1
triplet_loss = nn.TripletMarginLoss(margin=margin, p=4, eps=1e-3)


In [249]:
# Batch iterator
for video_batch, audio_batch, flow_ranks in dataloader:
    try:
        audio_optimizer.zero_grad()
        video_optimizer.zero_grad()
        total_loss = 0

        # create segments for each batch and compute embeddings for the segments
        # stack all the embeddings into single tensors
        batch_aud_embeddings, batch_vid_embeddings = get_batch_embeddings(video_model, audio_model, video_batch, audio_batch)

        
        # 1. Inter-modal loss
        inter_modal_loss = get_intermodal_loss(batch_vid_embeddings, batch_aud_embeddings, k=5, min_val=0)


        # 2. optical flow loss

        # this code finds the top and bottom ranked optical flow for a particular
        # video.  This is specified in flow ranks.  It then converts these indexes to 
        # their corresponding position in the stacked embeddings
        top_rank_idxs = []
        bottom_rank_idxs = []
        current_index = 0
        for ranks in flow_ranks:
            top_rank_idxs.append(current_index + np.argmin(ranks))
            bottom_rank_idxs.append(current_index + np.argmax(ranks))
            current_index += len(ranks)

        # Randomly choose num_flow_matching pairs to match (top_rank, top_rank, bottom rank)
        top_matching_samples = list(itertools.product(top_rank_idxs, top_rank_idxs, bottom_rank_idxs))
        top_matching_samples = [random.choice(top_matching_samples) for _ in range(num_flow_matching)]

        # Randomly choose num_flow_matching pairs to match (bottom, bottom, top_rank rank)
        bottom_matching_samples = list(itertools.product(bottom_rank_idxs, bottom_rank_idxs, top_rank_idxs))
        bottom_matching_samples = [random.choice(bottom_matching_samples) for _ in range(num_flow_matching)]

        of_loss_top = 0
        for anchor, pos, neg in top_matching_samples:
            of_loss_top += triplet_loss(batch_vid_embeddings[anchor], batch_vid_embeddings[pos], batch_vid_embeddings[neg])
            of_loss_top += triplet_loss(batch_aud_embeddings[anchor], batch_aud_embeddings[pos], batch_aud_embeddings[neg])

        of_loss_bottom = 0
        for anchor, pos, neg in bottom_matching_samples:
            of_loss_bottom += triplet_loss(batch_vid_embeddings[anchor], batch_vid_embeddings[pos], batch_vid_embeddings[neg])
            of_loss_bottom += triplet_loss(batch_aud_embeddings[anchor], batch_aud_embeddings[pos], batch_aud_embeddings[neg])

        loss = lambda1*inter_modal_loss + lambda2*of_loss_top + lambda3*of_loss_bottom
        loss.backward()
        audio_optimizer.step()
        video_optimizer.step()
        
    except Exception as e:
        # adding a wrapper just in case
        print(e)
    break

/var/folders/qb/2jqy46j9757g7m30b7cz_z040000gn/T/ipykernel_60234/3764631200.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  vf = torch.tensor(video_features[start_segment:end_segment,:])
/var/folders/qb/2jqy46j9757g7m30b7cz_z040000gn/T/ipykernel_60234/3764631200.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  af = torch.tensor(audio_features[start_segment:end_segment,:])


In [251]:
loss

tensor(1.1173, grad_fn=<AddBackward0>)